In [4]:
%pwd

u'/mnt/disk0/home/zhongrunxing/jupyter_code/automl_777'

In [30]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns


import os
import gc
import datetime
from random import choice
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings("ignore")


# prepare data

In [7]:
path = '/home/zhongrunxing/data/kaggle/homesite_quote/'
col_id_label = pd.read_csv(path + 'sample_submission.csv', nrows = 0)
col_id = col_id_label.columns[0]
col_label = col_id_label.columns[1]
print(col_id, col_label)

('QuoteNumber', 'QuoteConversion_Flag')


In [8]:
path = '/home/zhongrunxing/data/kaggle/give_me/stacking_test/'
col_id = 'Id'
col_label = 'Probability'

In [9]:
# read data
#n_rows = 10000
n_rows = None
train = pd.read_csv(path + 'train.csv', nrows = n_rows)
test = pd.read_csv(path + 'test.csv', nrows = n_rows)
print('read n_rows = ', train.shape)

('read n_rows = ', (150000, 93))


In [10]:
# prepare data
test_id = test[col_id]
test.drop([col_id], axis = 1, inplace = True)
train_label = train[col_label]
train.drop([col_label], axis = 1, inplace = True) 

if col_id in train.columns:
    train_id = train[col_id]
    train.drop([col_id], axis = 1, inplace = True)
if col_label in test.columns:
    test.drop([col_label], axis = 1, inplace = True)

n_train = train.shape[0]
train_test = pd.concat([train, test], axis = 0).reset_index(drop = True)

del train, test

In [11]:
if not os.path.exists(path + '/meta/'):
    os.mkdir(path + '/meta/')
test_id.to_csv(path + '/meta/test_id.csv', index = False, header = col_id)
train_label.to_csv(path + '/meta/train_label.csv', index = False, header = col_label)

In [12]:
print('train_test shape = ', train_test.shape)
print(train_test.dtypes.value_counts())

('train_test shape = ', (251503, 91))
float64    46
int64      45
dtype: int64


In [13]:
# preprocessing
print(train_test.isnull().sum().sum())
train_test.fillna(train_test.mean(), inplace=True)

0


In [14]:
col_cat = [col for col in train_test.columns if train_test.dtypes[col] == 'object']
for col in col_cat:
    le = LabelEncoder()
    train_test[col] = le.fit_transform(train_test[col])

In [15]:
train_fea = train_test[:n_train]
test_fea = train_test[n_train:]
del train_test

In [16]:
train_fea.reset_index(drop = True, inplace = True)
test_fea.reset_index(drop = True, inplace = True)

# generate meta feature

In [17]:
np.random.seed(777)
params_list = []
score_valid_list = []
for i in range(10):
    print('\n################################################## seed = ' + str(i))
#     params = {
#         'learning_rate': np.random.choice([0.001, 0.005, 0.01, 0.05, 0.1, 0.2]),
#         'num_leaves': np.random.choice([20, 31, 63, 100, 127, 200, 255, 300, 511]),
# #         'min_data': np.random.choice([300, 400, 450, 500, 550, 650]),
# #         'max_bin': np.random.choice([3, 5, 10, 12, 18, 20, 22]),
# #         'min_data_in_leaf':
#         'max_depth': np.random.choice([5, 6, 7, 8, 9]),     
#         'max_bin': np.random.choice([63, 127, 255, 511]),

#         'bagging_freq': np.random.choice([3,5,7]),
#         'bagging_fraction': np.random.choice([0.6, 0.7, 0.8, 0.9]),
#         'feature_fraction': np.random.choice([0.6, 0.7, 0.8, 0.9]),
# #         'lambda_l1':
# #         'lambda_l2':

#         'seed': i,
#         'num_threads': 7,
#         'task':'train', 
#         'boosting_type' : 'gbdt',
#         'objective': 'binary', 
#         'metric': 'auc'
#     } 
# use 3th feature
    params = {
        'task':'train', 
        'objective': 'binary',
        'metric': 'auc', 
        'seed': i,
        'n_jobs' : 15,

        'bagging_fraction': 0.7000000000000001,
        'bagging_freq': 11,
        'feature_fraction': 0.8,
        'learning_rate': 0.04349612990923269,
        'max_depth': -1,
        'min_child_weight': 0.5575853047609016,
        'num_leaves': 21,
        'reg_alpha': 29.847948107182233,
        'reg_lambda': 10.29487867938731
    }
    
    
    params_list.append(params)
    meta_train, meta_test, score_valid = generate_meta(train_fea, test_fea, train_label, params)
    score_valid_list.append(score_valid)
    pd.DataFrame(meta_train).to_csv(path + '/meta/meta_train_' + str(i)  + '.csv', index = False, header = None)
    pd.DataFrame(meta_test).to_csv(path + '/meta/meta_test_' + str(i)  + '.csv', index = False, header = None)
    del meta_train, meta_test

print('valid mean = {}'.format(np.mean(score_valid_list)))


################################################## seed = 0
cur time = 2018/11/22 13:37:29
cur time = 2018/11/22 13:38:29
whole valid auc = 0.8668073023427271
('best_iteration', [253, 331, 498, 469, 333], 376.8)
('train', array([0.87700734, 0.87719525, 0.87976857, 0.88070804, 0.87644209]), 0.8782242579354058)
('valid', array([0.86134959, 0.86838805, 0.86892716, 0.86469023, 0.87118389]), 0.866907784091784)

################################################## seed = 1
cur time = 2018/11/22 13:38:29
cur time = 2018/11/22 13:39:04
whole valid auc = 0.8668224457894322
('best_iteration', [289, 319, 379, 330, 352], 333.8)
('train', array([0.87834017, 0.87689234, 0.87788513, 0.87831451, 0.87677771]), 0.8776419735199015)
('valid', array([0.86078016, 0.86848143, 0.86940311, 0.864751  , 0.87101548]), 0.8668862362593286)

################################################## seed = 2
cur time = 2018/11/22 13:39:04
cur time = 2018/11/22 13:39:43
whole valid auc = 0.8666999340087186
('best_iteration', 

# stacking master

## load data

In [18]:
train_label = pd.read_csv(path + '/meta/train_label.csv')
col_label = train_label.columns[0]
train_label = train_label[col_label]

test_id = pd.read_csv(path + '/meta/test_id.csv')
col_id = test_id.columns[0]
test_id = test_id[col_id]

In [19]:
print(col_id, col_label)

('Id', 'Probability')


In [20]:
meta_train_fea = pd.DataFrame([])
meta_test_fea = pd.DataFrame([])
for i in range(10):
    meta_train_i = pd.read_csv(path + '/meta/meta_train_' + str(i)  + '.csv', header = None)
    meta_test_i = pd.read_csv(path + '/meta/meta_test_' + str(i)  + '.csv', header = None)
    meta_train_fea = pd.concat([meta_train_fea, meta_train_i], axis = 1)
    meta_test_fea = pd.concat([meta_test_fea, meta_test_i], axis = 1)

print(meta_train_fea.shape, meta_test_fea.shape)

((150000, 10), (101503, 10))


# add origin X

In [96]:
meta_train_fea = pd.concat([meta_train_fea, train_fea], axis = 1)
meta_test_fea = pd.concat([meta_test_fea, test_fea], axis = 1)
meta_train_fea.shape, meta_test_fea.shape

((150000, 101), (101503, 101))

In [21]:
print(meta_train_fea.shape, meta_test_fea.shape)

((150000, 10), (101503, 10))


# go go go

In [103]:
## raw X + 10 meta y， 0.869683
bagging_meta_train, bagging_meta_test = cv(meta_train_fea, meta_test_fea, train_label, 10, 5)


################################################## seed = 0
cur time = 2018/11/22 13:26:02
========================= seed = 0
current whole valid auc = 0.865964718673595
bagging whole valid auc = 0.865964718673595
('best_iteration', [213, 212, 224, 250, 212], 222.2)
('train', array([0.8685538 , 0.86747676, 0.8689759 , 0.86873289, 0.86800452]), 0.8683487746899736)
('valid', array([0.86508192, 0.87038634, 0.86431262, 0.86529288, 0.86839094]), 0.8666929377947705)

################################################## seed = 1
cur time = 2018/11/22 13:26:25
========================= seed = 1
current whole valid auc = 0.8662814381039007
bagging whole valid auc = 0.8662901306931327
('best_iteration', [256, 246, 290, 192, 224], 241.6)
('train', array([0.86848467, 0.86880863, 0.86753496, 0.86750598, 0.86965357]), 0.8683975623858773)
('valid', array([0.86744157, 0.86501793, 0.87088889, 0.86970775, 0.86169517]), 0.8669502620378522)

################################################## seed = 2
cur t

In [28]:
## 10 meta y， 0.869530
bagging_meta_train, bagging_meta_test = cv(meta_train_fea, meta_test_fea, train_label, 10, 5)


################################################## seed = 0
cur time = 2018/11/22 13:51:33
========================= seed = 0
current whole valid auc = 0.8663277840909676
bagging whole valid auc = 0.8663277840909676
('best_iteration', [132, 142, 147, 134, 142], 139.4)
('train', array([0.86869162, 0.86760342, 0.8690724 , 0.86868203, 0.86814198]), 0.868438292100276)
('valid', array([0.86574342, 0.87014857, 0.86459874, 0.86574936, 0.86821157]), 0.8668903318669235)

################################################## seed = 1
cur time = 2018/11/22 13:51:48
========================= seed = 1
current whole valid auc = 0.8662970497062845
bagging whole valid auc = 0.8664381694282394
('best_iteration', [167, 236, 130, 160, 102], 159.0)
('train', array([0.86862714, 0.86948573, 0.86735599, 0.86778436, 0.86957668]), 0.8685659779760393)
('valid', array([0.86718182, 0.86479851, 0.87125514, 0.87029127, 0.86164153]), 0.8670336512221304)

################################################## seed = 2
cur 

In [29]:
if not os.path.exists(path + '/result/'):
    os.mkdir(path + '/result/')
result = pd.DataFrame({col_id:test_id, col_label:bagging_meta_test})[[col_id, col_label]]
result.to_csv(path + '/result/stacking_baseline_only_meta.csv', index = False)

# model

In [2]:
### stack_master
def generate_meta(train_feature, test_feature, train_label, parmas):
    n_kfold = 5
    seed_kfold = 777
    print('cur time = ' + str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
    best_it = []
    score_list = []
    meta_train = np.zeros(len(train_feature))
    meta_test = np.zeros(len(test_feature))

    
    skf = StratifiedKFold(train_label, n_folds=n_kfold, shuffle=True, random_state=seed_kfold)
    for i,(train_idx,valid_idx) in enumerate(skf):
        ###########################
        if type(train_feature) == pd.DataFrame:
            X_train, y_train = train_feature.iloc[train_idx], train_label.iloc[train_idx]
            X_valid, y_valid = train_feature.iloc[valid_idx], train_label.iloc[valid_idx]
            X_test = test_feature
        elif type(train_feature) == np.ndarray:
            X_train, y_train = train_feature[train_idx], train_label[train_idx]
            X_valid, y_valid = train_feature[valid_idx], train_label[valid_idx]
            X_test = test_feature
        else:
            print('type error')
        ###########################

        dtrain = lgb.Dataset(X_train, y_train) 
        dvalid = lgb.Dataset(X_valid, y_valid, reference = dtrain)   
#         params = {
#             'task':'train', 
#             'objective': 'binary',
#             'metric': 'auc', 

#             'boosting_type':'gbdt',
#             'num_leaves': 31,
#             'learning_rate': 0.05,
#             'feature_fraction': 0.85,
#             'subsample':0.85,
#             'num_threads': 15,
#             'seed': 777,
#             'max_bin':127,
# #             'reg_alpha': 0.1,
# #             'reg_lambda': 0.1
#         }    
        model = lgb.train(params, 
                          dtrain, 
                          num_boost_round=3000,
                          valid_sets=[dtrain,dvalid],
                          verbose_eval=0, 
                          early_stopping_rounds=100)  

        best_it.append(model.best_iteration)
        score_list.append(model.best_score)

        pred_valid = model.predict(X_valid, model.best_iteration)
        pred_test = model.predict(X_test, model.best_iteration)
        
        meta_train[valid_idx] += pred_valid
        meta_test += pred_test
        gc.collect()
        
    print('cur time = ' + str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
    print("whole valid auc = " + str(roc_auc_score(train_label, meta_train)))
    print('best_iteration',best_it, np.mean(best_it))
    train_auc = np.array([i['training']['auc'] for i in score_list])
    valid_auc = np.array([i['valid_1']['auc'] for i in score_list])
    print('train', train_auc, np.mean(train_auc))
    print('valid', valid_auc, np.mean(valid_auc))
    
    
    meta_test = meta_test / n_kfold
    gc.collect()
    return meta_train, meta_test, np.mean(valid_auc)
    

In [27]:
#################
# StratifiedKFold, bagging
def cv(train_feature, test_feature, train_label, n_seed, n_kfold):
    col_fea = [col for col in train_feature.columns if col != 'auto_id']
    train_feature = train_feature[col_fea]
    test_feature = test_feature[col_fea]
    
    bagging_best_it = []
    bagging_score_list = []
    bagging_meta_train = np.zeros(len(train_feature))
    bagging_meta_test = np.zeros(len(test_feature))

    seed_list = range(n_seed)
    n_seed = len(seed_list)
    n_kfold = 5
    for si, seed in enumerate(seed_list):
        print('\n################################################## seed = ' + str(seed))
        print('cur time = ' + str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
        best_it = []
        score_list = []
        meta_train = np.zeros(len(train_feature))
        meta_test = np.zeros(len(test_feature))
        
        skf = StratifiedKFold(train_label, n_folds=n_kfold, shuffle=True, random_state=seed)
        for i,(train_idx,valid_idx) in enumerate(skf):
            #print('\n ##FOLD: ' + str(i))
            #print(len(train_idx), len(valid_idx))
            #print('cur time = ' + str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")))

            ###########################
            if type(train_feature) == pd.DataFrame:
                X_train, y_train = train_feature.iloc[train_idx], train_label.iloc[train_idx]
                X_valid, y_valid = train_feature.iloc[valid_idx], train_label.iloc[valid_idx]
                X_test = test_feature
            elif type(train_feature) == np.ndarray:
                X_train, y_train = train_feature[train_idx], train_label[train_idx]
                X_valid, y_valid = train_feature[valid_idx], train_label[valid_idx]
                X_test = test_feature
            else:
                print('type error')
            ###########################

            dtrain = lgb.Dataset(X_train, y_train) 
            dvalid = lgb.Dataset(X_valid, y_valid, reference = dtrain)   
            params = {
                'task':'train', 
                'objective': 'binary',
                'metric': 'auc', 

                'boosting_type':'gbdt',
                'num_leaves': 31,
                'learning_rate': 0.05,
                'feature_fraction': 0.85,
                'subsample':0.85,
                'num_threads': 15,
                'seed': seed,
                'max_bin':127,
    #             'reg_alpha': 0.1,
    #             'reg_lambda': 0.1
            }    
            
            # hypyeropt X + 10meta
            params = {
                'task':'train', 
                'objective': 'binary',
                'metric': 'auc', 
                'seed': seed,
                'n_jobs' : 15,
                
                'bagging_fraction': 0.7000000000000001,
                'bagging_freq': 16,
                'feature_fraction': 0.8,
                'learning_rate': 0.02461274520838658,
                'max_depth': 2,
                'min_child_weight': 5.061544739413494,
                'num_leaves': 200,
                'reg_alpha': 6.483593137740469,
                'reg_lambda': 11.22040214941558
            }
            
            # hypyeropt, only 10meta
            params = {
                'task':'train', 
                'objective': 'binary',
                'metric': 'auc', 
                'seed': seed,
                'n_jobs' : 15,
                
                'bagging_fraction': 0.9,
                'bagging_freq': 33,
                'feature_fraction': 0.6000000000000001,
                'learning_rate': 0.04413311077332326,
                'max_depth': 2,
                'min_child_weight': 6.498995431966556,
                'num_leaves': 37,
                'reg_alpha': 7.864559010265301,
                'reg_lambda': 1.6746319383189814
            }
            

            model = lgb.train(params, 
                              dtrain, 
                              num_boost_round=3000,
                              valid_sets=[dtrain,dvalid],
                              verbose_eval=0, 
                              early_stopping_rounds=100)  

            best_it.append(model.best_iteration)
            score_list.append(model.best_score)
            
            pred_valid = model.predict(X_valid, model.best_iteration)
            pred_test = model.predict(X_test, model.best_iteration)
            meta_train[valid_idx] += pred_valid
            meta_test += pred_test
            gc.collect()
            #print('cur time = ' + str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
        
        gc.collect()
        bagging_best_it += best_it
        bagging_score_list += score_list
        bagging_meta_train += meta_train
        bagging_meta_test += (meta_test / n_kfold)
        
        print('========================= seed = ' + str(seed))
        print("current whole valid auc = " + str(roc_auc_score(train_label, meta_train)))
        print("bagging whole valid auc = " + str(roc_auc_score(train_label, bagging_meta_train / (1.0 * (si + 1)))))
        print('best_iteration',best_it, np.mean(best_it))
        train_auc = np.array([i['training']['auc'] for i in score_list])
        valid_auc = np.array([i['valid_1']['auc'] for i in score_list])
        print('train', train_auc, np.mean(train_auc))
        print('valid', valid_auc, np.mean(valid_auc))
    
    
    # really bagging
    bagging_meta_train /= (1.0 * n_seed)
    bagging_meta_test /= (1.0 * n_seed)
    
    # average
    train_auc = np.array([i['training']['auc'] for i in bagging_score_list])
    valid_auc = np.array([i['valid_1']['auc'] for i in bagging_score_list])
    print('\n##############################################')
    print('### CV avergae result ###')
    print('cur time = ' + str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
    print('train auc = ' + str(train_auc.mean()))
    print('valid auc = ' + str(valid_auc.mean()))
    print('##############################################')
    print("bagging whole valid auc = " + str(roc_auc_score(train_label, bagging_meta_train / (1.0 * (si + 1)))))
   
    gc.collect()
    return bagging_meta_train, bagging_meta_test
    

In [4]:
import hyperopt
from hyperopt import hp, tpe, STATUS_OK, space_eval, Trials

config = {}
#config['mode'] = 'regression'
config['mode'] = 'classification'
params = {
        "objective": "regression" if config["mode"] == "regression" else "binary",
        "metric": "rmse" if config["mode"] == "regression" else "auc",
        "verbosity": -1,
        "seed": 1,
        "n_jobs":31
}

def merge_dict(d1, d2):
    tmp = d1.copy()
    tmp.update(d2)
    return tmp
def hyperopt_lightgbm(X, y, params, config):
    print('######################################################## begin hyperopt_lightgbm')
    print('####### cur time = ' + str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
     
    space = {
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.05),
        "max_depth": hp.choice("max_depth", [-1, 2, 3, 4, 5, 6]),
        "num_leaves": hp.choice("num_leaves", np.linspace(10, 200, 50, dtype=int)),
        "feature_fraction": hp.quniform("feature_fraction", 0.5, 1.0, 0.1),
        "bagging_fraction": hp.quniform("bagging_fraction", 0.5, 1.0, 0.1),
        "bagging_freq": hp.choice("bagging_freq", np.linspace(0, 50, 10, dtype=int)),
        "reg_alpha": hp.uniform("reg_alpha", 0, 30),
        "reg_lambda": hp.uniform("reg_lambda", 0, 30),
        "min_child_weight": hp.uniform('min_child_weight', 0.5, 10),
    }

    def objective(hyperparams):
        score_list = []
        skf = StratifiedKFold(train_label, n_folds=5, shuffle=True, random_state=777)
        for i,(train_idx,valid_idx) in enumerate(skf):
            X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            X_valid, y_valid = X.iloc[valid_idx], y.iloc[valid_idx]
            
            dtrain = lgb.Dataset(X_train, label=y_train)
            dvalid = lgb.Dataset(X_valid, label=y_valid)
    
            model = lgb.train(merge_dict(params, hyperparams), dtrain, 3000, dvalid, 
                              early_stopping_rounds=100, verbose_eval=-1)

            score = model.best_score["valid_0"][params["metric"]]
            score_list.append(score)
        
        score = np.mean(score_list)
        if config['mode'] == 'classification':
            score = -score
        return {'loss': score, 'status': STATUS_OK}

    trials = Trials()
    best = hyperopt.fmin(fn=objective, space=space, trials=trials, algo=tpe.suggest, max_evals=30, verbose=1,
                         rstate=np.random.RandomState(1))

    hyperparams = space_eval(space, best)
    print('########################################################## end hyperopt_lightgbm')
    print('####### cur time = ' + str(datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S")))
    print("{:0.4f} {}".format(trials.best_trial['result']['loss'], hyperparams))
    return hyperparams, best, trials

In [22]:
hyperparams, best, trials = hyperopt_lightgbm(meta_train_fea,  train_label, params, config)

######################################################## begin hyperopt_lightgbm
####### cur time = 2018/11/22 13:45:36
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[144]	valid_0's auc: 0.860646
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[112]	valid_0's auc: 0.868073
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[198]	valid_0's auc: 0.86929
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[288]	valid_0's auc: 0.864473
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[148]	valid_0's auc: 0.870667
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[190]	valid_0's auc: 0.860714
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[136]	valid_0's

Early stopping, best iteration is:
[604]	valid_0's auc: 0.864693
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[404]	valid_0's auc: 0.870757
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[330]	valid_0's auc: 0.861255
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[340]	valid_0's auc: 0.868376
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[418]	valid_0's auc: 0.869488
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[385]	valid_0's auc: 0.864729
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[192]	valid_0's auc: 0.870442
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[204]	valid_0's auc: 0.861112
Training until validation scores don't 

Early stopping, best iteration is:
[142]	valid_0's auc: 0.864856
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[103]	valid_0's auc: 0.870993
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[155]	valid_0's auc: 0.861143
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[127]	valid_0's auc: 0.868371
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[156]	valid_0's auc: 0.869641
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[178]	valid_0's auc: 0.864878
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[97]	valid_0's auc: 0.871028
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[104]	valid_0's auc: 0.860848
Training until validation scores don't i

In [24]:
trials_df = pd.DataFrame(trials.trials)
trials_df['auc'] = trials_df['result'].apply(lambda x: x['loss'])
trials_df = trials_df.sort_values(by = 'auc').reset_index(drop = True)

In [25]:
hyperparams

{'bagging_fraction': 0.9,
 'bagging_freq': 33,
 'feature_fraction': 0.6000000000000001,
 'learning_rate': 0.04413311077332326,
 'max_depth': 2,
 'min_child_weight': 6.498995431966556,
 'num_leaves': 37,
 'reg_alpha': 7.864559010265301,
 'reg_lambda': 1.6746319383189814}

In [26]:
trials_df.head() # 10 meta

,book_time,exp_key,misc,owner,refresh_time,result,spec,state,tid,version,auc
0,2018-11-22 05:49:48.345,None,"{u'tid': 25, u'idxs': {u'num_leaves': [25], u'...",None,2018-11-22 05:49:52.055,"{u'status': u'ok', u'loss': -0.86706556908}",None,2,25,0,-0.867066
1,2018-11-22 05:48:09.329,None,"{u'tid': 14, u'idxs': {u'num_leaves': [14], u'...",None,2018-11-22 05:48:19.823,"{u'status': u'ok', u'loss': -0.867057789732}",None,2,14,0,-0.867058
2,2018-11-22 05:48:30.885,None,"{u'tid': 16, u'idxs': {u'num_leaves': [16], u'...",None,2018-11-22 05:48:42.098,"{u'status': u'ok', u'loss': -0.867036331049}",None,2,16,0,-0.867036
3,2018-11-22 05:47:19.864,None,"{u'tid': 11, u'idxs': {u'num_leaves': [11], u'...",None,2018-11-22 05:47:31.525,"{u'status': u'ok', u'loss': -0.867014586128}",None,2,11,0,-0.867015
4,2018-11-22 05:49:52.089,None,"{u'tid': 26, u'idxs': {u'num_leaves': [26], u'...",None,2018-11-22 05:49:55.499,"{u'status': u'ok', u'loss': -0.86701213641}",None,2,26,0,-0.867012


In [100]:
trials_df.head() # X+10meta

,book_time,exp_key,misc,owner,refresh_time,result,spec,state,tid,version,auc
0,2018-11-22 05:23:38.142,None,"{u'tid': 26, u'idxs': {u'num_leaves': [26], u'...",None,2018-11-22 05:23:48.186,"{u'status': u'ok', u'loss': -0.866906448482}",None,2,26,0,-0.866906
1,2018-11-22 05:21:03.350,None,"{u'tid': 16, u'idxs': {u'num_leaves': [16], u'...",None,2018-11-22 05:21:19.344,"{u'status': u'ok', u'loss': -0.866720410891}",None,2,16,0,-0.866720
2,2018-11-22 05:22:13.890,None,"{u'tid': 21, u'idxs': {u'num_leaves': [21], u'...",None,2018-11-22 05:22:29.608,"{u'status': u'ok', u'loss': -0.866710523248}",None,2,21,0,-0.866711
3,2018-11-22 05:18:07.710,None,"{u'tid': 2, u'idxs': {u'num_leaves': [2], u're...",None,2018-11-22 05:18:18.272,"{u'status': u'ok', u'loss': -0.866664109561}",None,2,2,0,-0.866664
4,2018-11-22 05:20:36.841,None,"{u'tid': 14, u'idxs': {u'num_leaves': [14], u'...",None,2018-11-22 05:20:48.460,"{u'status': u'ok', u'loss': -0.866616888093}",None,2,14,0,-0.866617
